In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pickle
import pandas as pd

# Prepare data for mT5

get reference doc ids

In [ ]:
with open ('/content/drive/MyDrive/AQA-Data/tfidf_results1000.pickle', 'rb') as f:
  tfidf_results = pickle.load(f)

top_1 =[]
top_5 = []
top_10 = []
for k in tfidf_results.keys():
  counter = 0
  to_append_5 = []
  to_append_10 = []
  for value in tfidf_results[k]:
    if counter == 0:
      top_1.append(value[0])
    elif counter <= 4:
      to_append_5.append(value[0])
    elif counter <=9:
      to_append_10.append(value[0])
    else:
      break
    counter+=1
    
    top_5.append(to_append_5)
    top_10.append(to_append_10)

get reference doc text

In [ ]:
#urls = pd.read_csv('/content/reference_urls_collected.csv')  # Replace 'your_dataframe.csv' with the actual filename or path
content_list = []

for url in top_1:
    matching_row = urls[urls['URL'] == url]
    if not matching_row.empty:
        textual_content = matching_row['Text'].values[0]
        content_list.append((url, textual_content))
    else:
      content_list.append((url, 'Empty'))

get answer

In [ ]:
questions = pd.read_csv('/content/drive/MyDrive/AQA-Data/questions.csv')
answer_list = []

for question in tfidf_results.keys():
  matching_row = questions[questions['Question'] == question]
  if not matching_row.empty:
        answer = matching_row['Answer'].values[0]
        answer_list.append((question, answer))

Combine

In [ ]:
textual_content_list = [textual_content for url, textual_content in content_list]
answer_list = [answer for question, answer in answer_list]

text_summary_df = pd.DataFrame({'summary': answer_list, 'text': textual_content_list})
text_summary_df.head()

transform to hugging face

In [ ]:
from datasets import Dataset
import pyarrow as pa

arrow_table = pa.Table.from_pandas(text_summary_df)
arrow_dict = arrow_table.to_pydict()
text_summary = Dataset.from_dict(arrow_dict)

# Apply mT5 Part

In [ ]:
text_summary = text_summary.train_test_split(test_size=0.2, seed=42)

# Pre-process

In [ ]:
# Pre-process
from transformers import AutoTokenizer

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_summaries = text_summary.map(preprocess_function, batched=True)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate

In [ ]:
# Evaluate

import evaluate

rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


import numpy as np
from bert_score import score as bert_score
from rouge_score import rouge_scorer

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = scorer.score(decoded_preds, decoded_labels)

    # Calculate BERT scores
    P, R, F1 = bert_score(decoded_preds, decoded_labels, lang='en', verbose=False)

    # Calculate average prediction length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    avg_prediction_len = np.mean(prediction_lens)

    metrics = {
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure,
        'bert_precision': P.mean().item(),
        'bert_recall': R.mean().item(),
        'bert_f1': F1.mean().item(),
        'avg_prediction_len': avg_prediction_len
    }

    return metrics

# Train

In [ ]:
# Train
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="qa_1000_samples",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_summaries["train"],
    eval_dataset=tokenized_summaries["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()
#trainer.push_to_hub()

# Inference

In [ ]:
# Inference
from transformers import pipeline

summarizer = pipeline('summarization', model="natope/qa_1000_samples", device=0)
text_summary['test']['text']

summaries = []
for text in text_summary['test']['text']:
  summaries.append(summarizer(text, no_repeat_ngram_size=2, min_length=50, max_length=200))


gold_standard = text_summary['test']['summary'][:106]

generated_summaries = []
counter = 0
for i in range(len(summaries)):
  generated_summaries.append(summaries[counter][0]['summary_text'])
  counter+=1

# Calculate scores

In [ ]:
# Calculate scores
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import corpus_bleu
from sklearn.metrics import f1_score

generated_summaries = generated_summaries  # List of generated summaries
reference_summaries = gold_standard  # List of reference summaries

# Convert non-string elements to strings
generated_summaries = [str(summary) for summary in generated_summaries]
reference_summaries = [str(summary) for summary in reference_summaries]

# Compute BERT scores
P, R, F1 = bert_score(generated_summaries, reference_summaries, lang='nl', verbose=False)

# Compute ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
rouge_scores = []

for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    scores = scorer.score(generated_summary, reference_summary)
    rouge_scores.append(scores)

# Compute BLEU score
bleu_score = corpus_bleu([[ref.split()] for ref in reference_summaries], [gen.split() for gen in generated_summaries])

# Compute F1 score
f1 = f1_score(reference_summaries, generated_summaries, average='micro')  # Adjust 'average' parameter as needed

# Access the scores as needed
print("BERT Score - Precision:", P.mean().item())
print("BERT Score - Recall:", R.mean().item())
print("BERT Score - F1:", F1.mean().item())

avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print("ROUGE-1 (Average):", avg_rouge1)
print("ROUGE-2 (Average):", avg_rouge2)
print("ROUGE-L (Average):", avg_rougeL)

print("BLEU Score:", bleu_score)
print("F1 Score:", f1)